Importing some packages
===

In [12]:
import numpy as np
import scipy as sp
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

Next define where data is located relative to where the notebook is
=== 
For Windows replace / with \.

The cell reads all files in the folder and separates the name by _.

The last entry is used as condition. Whatever it might be :).

In [16]:
data_path='data_DQ/'
separator='_'
from os import listdir
from os.path import isfile, join
data_files = [f[:-4] for f in listdir(data_path) if (isfile(join(data_path, f)) and f[0]!='.' and f[-4:]=='.txt')]
conditions=[]
for i in data_files:
    conditions.append(i.split(separator)[-1]) #-1 denotes last entry

Give a series of peaks for Apo and ligand
===

Define m/z of Apo and Ligand peaks and an uncertainty (if data is collected at different times peak position might vary a bit. We are taking the maximum intensity within peak+- uncertainty as intensity for the peak.

In [17]:
Apo=[4635,5215,5960,6953,8344]#4172,
Lig=[4665,5249,5990,6998,8397]#4199,
uncert=1.0

Let the script run!
===

results is an numpy array with condition, apo, ligand as columns. Ratio has the columns condition, lig/apo.

In [18]:
results=[]
for i_file in range(len(data_files)):
    this_res=[]
    this_res.append(float(conditions[i_file]))
    file_path=data_path+data_files[i_file]+'.txt'
    mzint=pd.read_csv(file_path, skiprows=10, delimiter='\t').to_numpy().T
    int_Apo=0.0
    list_Apo=[]
    int_Lig=0.0
    list_Lig=[]
    for i in range(len(Apo)):
        lowupp_mz=[]
        lowupp_mz.append(np.argmin(np.abs(mzint[0]-(Apo[i]-uncert))))
        lowupp_mz.append(np.argmin(np.abs(mzint[0]-(Apo[i]+uncert))))
        list_Apo.append(lowupp_mz)
        int_Apo+=np.max(mzint[1][lowupp_mz[0]:lowupp_mz[1]])
        lowupp_mz=[]
        lowupp_mz.append(np.argmin(np.abs(mzint[0]-(Lig[i]-uncert))))
        lowupp_mz.append(np.argmin(np.abs(mzint[0]-(Lig[i]+uncert))))
        list_Lig.append(lowupp_mz)
        int_Lig+=np.max(mzint[1][lowupp_mz[0]:lowupp_mz[1]])
        weight_Apo=int_Apo/(int_Apo+int_Lig)
        weight_Lig=int_Lig/(int_Apo+int_Lig)
    this_res.append(weight_Apo)
    this_res.append(weight_Lig)
    results.append(this_res)
results=np.sort(np.array(results).view('f8,f8,f8'), order=[], axis=0).view(np.float)
results=results.T
ratio=[]
ratio.append(results[0])
ratio.append(results[2]/results[1])
ratio=np.array(ratio)

Visualisation using Bokeh
===
See https://bokeh.org for more information.

In [19]:
output_notebook()

# create a new plot with a title and axis labels
p = figure(title="Time Series of Ligand Binding", x_axis_label='Time in Minutes', y_axis_label='Ratio of Ligand Peaks / Apo Peaks')

# add a line renderer with legend and line thickness
p.scatter(ratio[0], ratio[1], line_width=2)
# show the results
show(p)

Loading BokehJS ...

Save ratio and results as csv files
===

In [10]:
np.savetxt('ratio.csv', ratio.T)
np.savetxt('results.csv', results.T)